## tensorflow tf.argmax() 用法 例子

argmax()官方文档如下：
 >tf.argmax(input, dimension, name=None) 
Returns the index with the largest value across dimensions of a tensor. 
Args: 
1. input: A Tensor. Must be one of the following types: float32, float64, int64, int32, uint8, int16, int8, complex64, qint8, quint8, qint32. 
2. dimension: A Tensor of type int32. int32, 0 <= dimension < rank(input). Describes which dimension of the input Tensor to reduce across. For vectors, use dimension = 0. 
3. name: A name for the operation (optional). 
4. Returns: A Tensor of type int64.

* dimension=0 按列找 
* dimension=1 按行找 
* tf.argmax()返回最大数值的<font color=red >**下标**</font>

通常和tf.equal()一起使用，计算模型准确度     
`
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
`

这里返回一个布尔数组。为了计算我们分类的准确率，我们将布尔值转换为浮点数来代表对、错，然后取平均值。例如：[True, False, True, True]变为[1,0,1,1]，计算出平均值为0.75。


In [25]:
import tensorflow as tf
a = tf.constant([1.,2.,3.,0.,9.,])
b = tf.constant([[1,2,3],[3,2,1],[4,5,6],[6,5,4]])
with tf.Session() as sess:
  m1 = sess.run(tf.argmax(a, 0));
  print(m1)
  m2 = sess.run(tf.argmax(a));
  print(m2);

4
4


In [26]:
with tf.Session() as sess:
 print(sess.run(tf.argmax(b, 0)))
 print(sess.run(tf.argmax(b)))
 print(b)

[3 2 2]
[3 2 2]
Tensor("Const_19:0", shape=(4, 3), dtype=int32)


In [27]:
with tf.Session() as sess:
 print(sess.run(tf.argmax(b, 1)))


[2 0 2 0]


注意此处的结果 <font color=red> 下标 </font>

## Tensorboard

tensorboard是通过读取events files来显示的，events file则包含在图计算时生成的summary data。

`1 tf.summary.scalar('mean', mean)
 2 tf.summary.histogram('histogram', var)
`

1.summary data
某个summary操作原型：
tf.summary.scalar(name, tensor, collections=None)
这些summary的操作都是对某个tensor产生单个的summary protocol buffer，是一种能被tensorboard读取的格式。Summaries provide a way to export condensed information about a model, which is then accessible in tools such as TensorBoard.
summary的操作对于整个图来说相当于是外设，因为tensorflow是由结果驱动的，而图的结果并不依赖于summary操作，所以summary操作需要被run。整个图经常需要检测许许多多的值，也就是许多值需要summary operation，一个个去run来启动太麻烦了，tensorflow为我们提供了这个函数：
merged=tf.summary.merge_all(key=tf.GraphKeys.SUMMARIES)
#Merges all summaries collected in the default graph.
 

把图中所有的summary数据合并在一起，一个run就能启动所有的summary operations。像这样：
summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
 

btw也有单独merge函数。
tensorboard是从磁盘中读events files的。tensorflow用一个tensorflow.summary.FileWriter将图 运行得到的summary 数据写到磁盘里。FileWriter的构造器包含一个logdir也就是summary data写到的位置，以及可选的图。
train_writer = tf.summary.FileWriter(FLAGS.summaries_dir +'/train',
sess.graph)
 

把每一次run的信息和得到的数据加到writer里面
1 train_writer.add_run_metadata(run_metadata, 'step%03d'% i)
2 train_writer.add_summary(summary, i)
 

最后把writer关掉，大功告成。
1 train_writer.close()


https://www.cnblogs.com/wyh1993/p/1f142f2d7daab0a8d74907f1d44effe7.html

## tf.scalar tf.summary

In [7]:
import tensorflow as tf
import numpy as np

cell = tf.nn.rnn_cell.BasicRNNCell(num_units=128) # state_size = 128
print(cell.state_size) # 128

inputs = tf.placeholder(np.float32, shape=(32, 100)) # 32 是 batch_size
h0 = cell.zero_state(32, np.float32) # 通过zero_state得到一个全0的初始状态，形状为(batch_size, state_size)
# output, h1 = cell.call(inputs, h0) #调用call函数
cell.call(inputs,h0)
# print(h1.shape) # (32, 128)

128


AttributeError: 'BasicRNNCell' object has no attribute '_kernel'